<a href="https://colab.research.google.com/github/mrohith29/PhysiSolve/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step - 1
### spplitting the dataset into training, testing and evaluation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

with open(r"/content/drive/MyDrive/dataset/high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

train_size = int(0.7 * len(data))
test_size = int(0.15 * len(data))
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
eval_data = data[train_size + test_size:]

# Save train, test and eval sets
with open(r"/content/drive/MyDrive/dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/eval.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, indent=4)

import json
import random

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

with open(r"/content/drive/MyDrive/dataset/high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

train_size = int(0.70 * len(data))
test_size = int(0.15 * len(data))
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
eval_data = data[train_size + test_size:]

# Save train, test, and evaluation sets
with open(r"/content/drive/MyDrive/dataset/train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

with open(r"/content/drive/MyDrive/dataset/eval.json", "w", encoding="utf-8") as f:
    json.dump(eval_data, f, indent=4)

print(f"Dataset split into {len(train_data)} training, {len(test_data)} testing, and {len(eval_data)} evaluation samples.")

Dataset split into 280 training, 60 testing, and 60 evaluation samples.


In [ ]:
print(test_data[0])
print(test_data[3])

{'id': 261, 'question': 'A 10 Ω resistor is connected across a 15 V battery. What is the current flowing through the resistor?', 'subject': 'Electrostatics and Current Electricity', 'choices': ['0.5 A', '1 A', '1.5 A', '2 A'], 'answer': 'C', 'explanation': 'I = V/R = 15/10 = 1.5 A', 'dataset': 'high_school_physics'}
{'id': 338, 'question': 'A force of 80 N is applied to a 8 kg block. What is the acceleration of the block?', 'subject': 'Mechanics', 'choices': ['5 m/s²', '10 m/s²', '15 m/s²', '20 m/s²'], 'answer': 'B', 'explanation': 'F = ma => a = F/m = 80/8 = 10 m/s²', 'dataset': 'high_school_physics'}


### Few-Shot Evaluation of Flan-T5 from Hugging Face

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

def evaluate_model(model, dataset):
    correct = 0
    total = len(dataset)

    for item in dataset:
        question = item["question"]
        idx = ord(item["answer"])-ord("A")
        correct_answer = item["choices"][idx]

        prompt = f"Give me the final answer without any explaination, just the couple of words with units that directly show the answer for the Question: {question} with Choices: {', '.join(item['choices'])} Answer:"
        prediction = model(prompt, max_length=20, truncation=True)[0]["generated_text"]

        # print(correct_answer, prediction, item["id"])
        if correct_answer in prediction:
          correct += 1

    accuracy = (correct / total) * 100
    return accuracy

zero_shot_accuracy = evaluate_model(qa_pipeline, test_data)
print(f"Zero-Shot Accuracy: {zero_shot_accuracy:.2f}%")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Zero-Shot Accuracy: 33.33%


### Fine-Tuning Flan-T5 using LoRA

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pickle

# Load model and tokenizer
model_name = "google/flan-t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Create custom dataset
class PhysicsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Question: {item['question']} Choices: {', '.join(item['choices'])} Answer:"
        idx = ord(item["answer"][0])-ord("A")
        target = item["choices"][idx]

        encodings = self.tokenizer(prompt, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze(),
            "labels": target_encodings["input_ids"].squeeze(),
        }

# Load json files
import json
train_file_path = "/content/drive/MyDrive/dataset/train.json"  # Update with your correct path
with open(train_file_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Prepare datasets
train_dataset = PhysicsDataset(train_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

# Train model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# --- Save the trained model as a pickle file ---
model_file_path = "/content/drive/MyDrive/dataset/trained_model.pkl"
with open(model_file_path, "wb") as f:
    pickle.dump(model, f)
    print(f"The trained model was stored in {model_file_path}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.EPOCH but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 